In [6]:
!pip install tokenizers==0.5.2

     |████████████████████████████████| 3.7MB 3.1MB/s 
  Found existing installation: tokenizers 0.7.0
    Uninstalling tokenizers-0.7.0:
      Successfully uninstalled tokenizers-0.7.0


In [7]:
!pip install simpletransformers

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import os
import re
import pandas as pd
import numpy as np

In [0]:
train = pd.read_csv('/content/gdrive/My Drive/Competitions/Zindi_VaccinateOrNot/Train.csv')
test = pd.read_csv('/content/gdrive/My Drive/Competitions/Zindi_VaccinateOrNot/Test.csv')
sub = pd.read_csv('/content/gdrive/My Drive/Competitions/Zindi_VaccinateOrNot/SampleSubmission.csv')

In [0]:
train['label'] = train['label'].fillna(0)
test['safe_text'] = test['safe_text'].fillna('NA')

#train[train['safe_text']=='1']
train.loc[train['safe_text']=='1','label'] = 1

In [0]:
train = train[['safe_text','label']]
test = test[['safe_text']]

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_cv =  train_test_split(train, test_size=0.20, random_state=42, stratify=train['label'])

In [0]:
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import accuracy_score, log_loss, mean_squared_error

In [0]:
model = ClassificationModel('roberta', 'roberta-base', num_labels=1, use_cuda=True, 
                            args={'reprocess_input_data': False, 
                                  'fp16': False, 
                                  'regression': True, 
                                  'overwrite_output_dir': True, 
                                  'num_train_epochs': 1
                                  }
                            )

In [46]:
model.train_model(X_train)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:247: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running loss: 0.443277

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.280388



In [47]:
scores1, model_outputs, wrong_predictions = model.eval_model(X_cv, mse=mean_squared_error)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:664: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [48]:
scores1

{'eval_loss': 0.2784259440845799, 'mse': 0.27939504}

In [49]:
np.sqrt(mean_squared_error(X_cv['label'], model_outputs))

0.5285782997505167

In [50]:
predictions, raw_output = model.predict(test['safe_text'], multi_label=False)

In [51]:
raw_output.shape

(5177,)

In [52]:
sub['label'] = raw_output
sub.head(3)

,tweet_id,label
0,00BHHHP1,0.451747
1,00UNMD0E,0.179795
2,01AXPTJF,-0.001022


In [53]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

def create_download_link(df, title = "Download CSV file", filename = "submission.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(sub)